In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx

In [ ]:
# score_type can be 'ORF' or 'CRISPR'
score_type = 'ORF'
#score_type = 'CRISPR'

dfscores = pd.read_csv('data/'+score_type.lower()+'_scores_merged.tsv',sep='\t')
#dfscores = pd.read_csv('/mnt/c/Datas/MorphMap/crispr_scores_merged.tsv',sep='\t')
#label = 'CRISPR'
label = score_type
display(dfscores)

In [ ]:
# Example : we select all connections between a gene and other genes

gene_name = 'YAP1'

dfscores_filtered = dfscores.loc[((dfscores['GENE1']==gene_name)|(dfscores['GENE2']==gene_name))&(dfscores['ABS_'+label+'_SIM']>0.4)]
display(dfscores_filtered)
dfscores_filtered.to_csv('data/'+gene_name+'_'+label+'_scores.tsv',sep='\t',index=False)
conns = list(set(list(dfscores_filtered['GENE1'])+list(dfscores_filtered['GENE2'])))
conns.sort()
for g in conns:
    if not g==gene_name:
        print(g)

In [ ]:
# Create the clustergram (conns can be replaced with any list of genes)

genes1 = list(conns)
dfscores_subm = dfscores.loc[((dfscores['GENE1'].isin(genes1))&(dfscores['GENE2'].isin(genes1)))]
#score_name = 'ABS_'+label+'_SIM'
score_name = label+'_SIM'
#score_name = 'unsupervised_max'
#score_name = 'gene_bp'
#score_name_annot = 'gene_bp'
#score_name_annot = 'ABS_'+label+'_SIM'
score_name_annot = label+'_SIM'
score_name_annot = 'unsupervised_max'
scores_dict = {}
scores_dict_annot = {}
for i,row in dfscores_subm.iterrows():
    scores_dict[(row['GENE1'],row['GENE2'])] = row[score_name]
    scores_dict[(row['GENE2'],row['GENE1'])] = row[score_name]
    scores_dict_annot[(row['GENE1'],row['GENE2'])] = row[score_name_annot]
    scores_dict_annot[(row['GENE2'],row['GENE1'])] = row[score_name_annot]

display(dfscores_subm)
sc_matrix = np.zeros([len(genes1),len(genes1)])
sc_matrix_annot = np.zeros([len(genes1),len(genes1)])
for i,gi in enumerate(genes1):
    for j,gj in enumerate(genes1):
        if i==j:
            sc_matrix[i,j] = 1.0
            sc_matrix_annot[i,j] = 1.0            
        else:
            sc_matrix[i,j] = scores_dict[(gi,gj)]
            sc_matrix[j,i] = scores_dict[(gj,gi)]
            sc_matrix_annot[i,j] = scores_dict_annot[(gi,gj)]
            sc_matrix_annot[j,i] = scores_dict_annot[(gj,gi)]

df = pd.DataFrame(data=sc_matrix,columns=genes1)
df.index = genes1
plt.rcParams['figure.dpi'] = 300
sns.set(font_scale = 0.8)
sns.clustermap(df,vmin=-1,vmax=1,annot=sc_matrix_annot,xticklabels=True,annot_kws={"size": 5},cmap='vlag')
plt.title(score_name)